In [ ]:
import pandas as pd
import networkx as nx  


In [ ]:
file = 'assoSongExp.h5'


inputdf = pd.read_hdf(file, key='asso', mode='r')

In [ ]:
G = nx.from_pandas_edgelist(inputdf, 'initiator', 'object', ['weight'])


In [ ]:
centrality = nx.eigenvector_centrality(G ,max_iter=500, weight = 'weight' )

In [ ]:
result = pd.DataFrame(data = centrality,index=[0]).T

In [ ]:
result.to_csv('eigenvectorExp.csv',header = None)

In [ ]:
result